# 2.3 将令牌转换为令牌 ID

在前一小节中，我们将伊迪丝·沃顿的一个短篇故事分割成了单个令牌(token)。
在本节中，我们将把这些令牌(token)从 Python 字符串转换成整数表示，生成所谓的令牌 ID。
这种转换是在将令牌 ID 转换成嵌入向量之前的中间步骤。

为了将先前生成的令牌(token)映射到令牌(token) ID，我们首先需要构建一个所谓的词汇表。
这个词汇表定义了我们如何将每个独特的词和特殊字符映射到一个独特的整数，如图 2.6 所示。

**图 2.6 我们通过将训练数据集中的整个文本分割成单个令牌(token)来构建词汇表。
这些单独的令牌(token)随后按字母顺序进行排序，并移除重复的令牌(token)。
然后，将这些独特的令牌(token)聚集成一个词汇表，该词汇表定义了从每个独特令牌(token)到一个独特整数值的映射。
所展示的词汇表为了说明目的故意保持较小，并且为了简化没有包含标点符号或特殊字符。**

fig2.6

在前一节中，我们对伊迪丝·沃顿的短篇故事进行了分词，并将其赋值给一个名为“preprocessed”的 Python 变量。
现在，让我们创建一个包含所有独特令牌(token)的列表，并按字母顺序排序以确定词汇表的大小：

In [1]:
import re
import requests

url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
response = requests.get(url)
raw_text = response.text
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if
item.strip()]

all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(vocab_size)

1159


在通过上述代码确定词汇表有1159个单词后，我们创建词汇表并打印其前50个单词用来展示说明。

### 代码示例 2.2 创建词汇表

In [2]:
vocab = {token:integer for integer,token in
enumerate(all_words)}
for i, item in enumerate(vocab.items()):
 print(item)
 if i > 50:
  break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Carlo;', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Destroyed', 30)
('Devonshire', 31)
('Don', 32)
('Dubarry', 33)
('Emperors', 34)
('Florence', 35)
('For', 36)
('Gallery', 37)
('Gideon', 38)
('Gisburn', 39)
('Gisburns', 40)
('Grafton', 41)
('Greek', 42)
('Grindle', 43)
('Grindle:', 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)
('He', 50)
('Her', 51)


输出结果如下：

('!', 0) \
('"', 1) \
("'", 2) \
... \
('Has', 49) \
('He', 50)

正如我们从上面输出结果中看到的，这个字典包含了与独特整数标签相关联的单个令牌(token)。
我们的下一个目标是应用这个词汇表，将新文本转换为令牌(token)ID，如图 2.7 所示。

**图 2.7 从一个新的文本样本开始，我们对文本进行分词，并使用词汇表将文本令牌(token)转换为令牌(token) ID。
这个词汇表是基于整个训练集构建的，并且可以应用于训练集本身及任何新的文本示例。
接下来所展示的词汇表为了简化起见，将不包含标点符号或特殊字符。**

fig2.7

在本书的后面，当我们想要将大型语言模型（LLM）的输出从数字转换回文本时，我们也需要一种方法将令牌(token) ID 转换回文本。
为此，我们可以创建一个词汇表的逆向版本，将令牌(token) ID 映射回对应的令牌(token)标记。

让我们通过 Python 来实现一个完整的分词器类，其中包括一个 编码(encode) 方法，该方法将文本分割成令牌(token)，并通过词汇表执行字符串到整数的映射以生成令牌(token) ID。
此外，我们还实现一个 解码(decode) 方法，该方法执行整数到字符串的反向映射，将令牌(token) ID 转换回文本。

这个分词器实现的代码如代码示例 2.3 所示：

### 代码示例2.3 实现一个简单的文本分词器

In [3]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab #A
        self.int_to_str = {i:s for s,i in vocab.items()} #B
 
    def encode(self, text): #C
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed
if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
 
    def decode(self, ids): #D
        text = " ".join([self.int_to_str[i] for i in ids])  
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #E
        return text

使用上述的 SimpleTokenizerV1 Python 类，我们现在可以通过现有的词汇表实例化新的分词器对象，然后我们可以使用它来编码和解码文本，如图 2.8 所示。

**图 2.8 分词器实现共有两个常用方法：一个是编码方法，另一个是解码方法。
编码方法接收样本文本，将其分割为单独的标记，并通过词汇表将这些标记转换为标记 ID。
解码方法接收令牌(token) ID，将它们转换回文本令牌(token)，并将这些文本令牌(token)连接成自然文本。**

fig2.8

让我们从 SimpleTokenizerV1 类实例化一个新的分词器对象，并使用它来对伊迪丝·沃顿的一段短篇故事进行分词，接下来在实践中尝试一下：

In [4]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," Mrs. Gisburn
said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 5, 1, 69, 7, 39, 873, 1136, 773, 812, 7]


上述代码打印出以下代码的令牌(token) ID：

[1, 58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 5, 1, 69, 7,
39, 873, 1136, 773, 812, 7]

接下来，让我们看看是否可以使用解码方法将这些令牌(token) ID 转换回文本：

In [5]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

这将输出以下文本：

'" It\' s the last he painted, you know," Mrs. Gisburn said
with pardonable pride.'

根据上面的输出，我们可以看到解码方法成功地将令牌(token)ID 转换回原始文本。

到目前为止，一切顺利。
这样，我们就构建了一个分词器，能够根据训练集中的一个片段对文本进行分词和解码。
现在，让我们将其应用于训练集中未包含的一个新文本示例：

In [6]:
text = "Hello, do you like tea?"
tokenizer.encode(text)

KeyError: 'Hello'

执行上述代码将导致以下错误：

...
KeyError: 'Hello'

问题在于单词 "Hello" 没有出现在《判决》这部短篇故事中。
所以，这个单词不包含在我们之前构建的词汇表中。
这突显出在处理大型语言模型（LLM）时，考虑使用大规模且多样化的训练集以扩展词汇表的重要性。

在下一节中，我们将进一步测试分词器处理包含未知词汇的文本，
并且我们还将讨论可以用来在训练过程中为大型语言模型（LLM）提供更多上下文的额外特殊令牌(token)。